In [1]:
!pip install -q -U datasets==2.13.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 3.0 MB/s eta 0:00:00


In [2]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.nn.functional import log_softmax, softmax

In [3]:
model_name = "cnicu/t5-small-booksum"
teacher_model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
train_dataset = load_dataset("kmfoda/booksum", split="train")
train_dataset = train_dataset.select_columns(
    [
        "summary_length",
        "summary_text",
        "chapter",
    ]
)
train_dataset = train_dataset.filter(
    lambda x: x["chapter"] is not None and x["summary_text"] is not None
)

In [5]:
TEST_DATA = """\
The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.
"""

In [6]:

def test_model(prefix, model):
    model.eval()

    input_ids = tokenizer.encode(
        TEST_DATA,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512,
        add_special_tokens=False,
    )

    generated_ids = model.generate(input_ids, max_length=120)[0]
    print(
        prefix
        + " "
        + tokenizer.decode(
            generated_ids,
            skip_special_tokens=True,
            remove_invalid_values=True,
        )
    )


In [7]:
def tokenize_function(example):
    inputs = tokenizer(
        example["chapter"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )
    targets = tokenizer(
        example["summary_text"],
        padding="max_length",
        truncation=True,
        max_length=256,
        return_tensors="pt",
    )

    return {
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
        "labels": targets.input_ids,
    }

In [8]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
train_dataset = train_dataset.remove_columns(
    ["chapter", "summary_text", "summary_length"]
)


train_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"]
)

Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

In [13]:
config = teacher_model.config
config.num_layers = 2
config.d_model = 128
config.d_ff = 512
config.d_kv = 64

In [ ]:
student_model = T5ForConditionalGeneration(config)
student_model

In [15]:
# Hyperparameters
learning_rate = 0.005
batch_size = 16
num_epochs = 13
temperature = 20
alpha = 0.7

In [16]:
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

In [17]:
optimizer = torch.optim.AdamW(student_model.parameters(), lr=learning_rate)

In [18]:
def calculate_loss(student_outputs, teacher_outputs, labels):
    s_logits = student_outputs.logits
    t_logits = teacher_outputs.logits

    vocab_size = s_logits.size(-1)
    ce_logits = s_logits.view(-1, vocab_size)
    ce_labels = labels.view(-1)
    ce_loss = torch.nn.functional.cross_entropy(ce_logits, ce_labels)
    student_log_probs = log_softmax(s_logits.view(-1, vocab_size) / temperature, dim=-1)
    teacher_probs = softmax(t_logits.view(-1, vocab_size) / temperature, dim=-1)

    distill_loss = torch.nn.functional.kl_div(
        student_log_probs, teacher_probs, reduction="batchmean"
    )
    loss = (1 - alpha) * ce_loss + (
        alpha * temperature**2 / batch_size**2
    ) * distill_loss

    return loss

In [ ]:

# Training Loop
for epoch in range(num_epochs):
    loss_value = 0

    test_model("Before epoch " + str(epoch), student_model)
    student_model.train()

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}")

    for batch in progress_bar:
        optimizer.zero_grad()

        batch = dict([(k, v) for k, v in batch.items()])

        # Forward pass through the teacher model
        with torch.no_grad():
            teacher_outputs = teacher_model(**batch)

        # Forward pass through the student model
        student_outputs = student_model(**batch)
        assert student_outputs.logits.size() == teacher_outputs.logits.size()
        loss = calculate_loss(student_outputs, teacher_outputs, batch["labels"])
        # Backpropagation
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix(loss_value=loss.item())


student_model.save_pretrained("distilled-t5-small")

Before epoch 0 WaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWaterWater


Epoch 0:   3%|▎         | 19/600 [10:00<5:15:22, 32.57s/it, loss_value=2.31]